In [13]:
from textblob import TextBlob
import itertools

In [11]:
dict1 = {"H": "I sais"}

data/                 name_dict.pyc         transcript_parser.py
name_dict.py          nltk practice.ipynb


In [ ]:
aggregate_dict = {}
for d in relevant_debates:
    for k, v in d.iteritems():
        if aggregate_dict.has_key(k):
            aggregate_dict[k] = aggregate_dict[k] + v
        else:
            aggregate_dict[k] = v

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(articles_df['content'])